# RegionBox: A Simple Way to Work with 3D Image Regions

## Overview

The `RegionBox` class helps define, manipulate, and extract regions from
3D images. If you're working with medical images (like CT or MRI scans),
you'll often need to isolate specific areas—whether for visualization,
processing, or deep learning tasks.

With `RegionBox`, you can:

- Define a 3D box with minimum and maximum coordinates.
- Expand, pad, or adjust the box.
- Extract regions from images.
- Create a `RegionBox` from segmentation masks.



## Getting Started

To create a `RegionBox`, all you need are two 3D points:  
the minimum (corner closest to the origin) and the maximum (opposite corner).


In [2]:
from imgtools import Coordinate3D, RegionBox
from rich import print

# Define a box from (5,5,5) to (10,10,10)
box = RegionBox(Coordinate3D(5, 5, 5), Coordinate3D(10, 10, 10))

print(box)

RegionBox(
        min=Coordinate3D(x=5, y=5, z=5),
        max=Coordinate3D(x=10, y=10, z=10)
        size=Size3D(w=5, h=5, d=5)
)

## Working with RegionBox

### Expanding the Box

Sometimes, you need to make the box bigger—whether to ensure  
it contains enough context or meets a minimum required size.

In [3]:
# Expand the box symmetrically by 5 units in all directions
expanded_box = box.pad(5)
print(expanded_box)

RegionBox(
        min=Coordinate3D(x=0, y=0, z=0),
        max=Coordinate3D(x=15, y=15, z=15)
        size=Size3D(w=15, h=15, d=15)
)

By default, the box expands equally in all directions by `padding` voxels.

If you only want to expand in one direction (the max side), use the `BoxPadMethod.END` option:


In [9]:
from imgtools import BoxPadMethod

end_padded_box = box.pad(5, method=BoxPadMethod.END)
print(end_padded_box)

RegionBox(
        min=Coordinate3D(x=5, y=5, z=5),
        max=Coordinate3D(x=15, y=15, z=15)
        size=Size3D(w=10, h=10, d=10)
)

### Creating a RegionBox from a Mask

If you have a segmentation mask (where a structure of interest is labeled),  
you can automatically create a `RegionBox` around it.


In [15]:
from imgtools.datasets import example_data

mask = example_data()['mask']

# Create a box around the mask's bounding box
bbox_region = RegionBox.from_mask_bbox(mask)
print(bbox_region)

RegionBox(
        min=Coordinate3D(x=45, y=21, z=67),
        max=Coordinate3D(x=55, y=38, z=84)
        size=Size3D(w=10, h=17, d=17)
)

You can create a `RegionBox` from the centroid of the mask.

This will create a `RegionBox` of size 0, where the min and max points are the same.

However, you can expand the box to include more context around the mask.

In [13]:
# Create a box around the mask's centroid
centroid_box = RegionBox.from_mask_centroid(mask)
print(centroid_box)

expanded_centroid_box = centroid_box.expand_to_min_size(10)
print(expanded_centroid_box)

RegionBox(
        min=Coordinate3D(x=50, y=29, z=76),
        max=Coordinate3D(x=50, y=29, z=76)
        size=Size3D(w=0, h=0, d=0)
)

RegionBox(
        min=Coordinate3D(x=45, y=24, z=71),
        max=Coordinate3D(x=55, y=34, z=81)
        size=Size3D(w=10, h=10, d=10)
)

### Cropping an Image

Once you have a `RegionBox`, you can crop an image to the exact region.  
This is useful when extracting specific anatomical structures.


In [16]:
import SimpleITK as sitk

# Load an example image (100x100x100 voxel CT scan)
image = example_data()['duck']

cropped_image = bbox_region.crop_image(image)
print(cropped_image.GetSize())

(10, 17, 17)

You can also crop both the image and the mask at the same time.

In [18]:
cropped_image, cropped_mask = bbox_region.crop_image_and_mask(image, mask)

print(cropped_image.GetSize())
print(cropped_mask.GetSize())

(10, 17, 17)

(10, 17, 17)

In [24]:
# Perform multiple steps together
cropped_image, cropped_mask = (
  RegionBox.from_mask_centroid(mask)
    .expand_to_min_size(20)
    .crop_image_and_mask(image, mask)
)

print(cropped_image.GetSize())
print(cropped_mask.GetSize())

(20, 20, 20)

(20, 20, 20)

## Summary

The `RegionBox` class makes it easy to define, manipulate, and extract  
regions from 3D images. Whether you're cropping scans or defining  
analysis regions, it provides a simple and flexible interface.

Key features:
- Define regions with `RegionBox(min, max)`.
- Expand regions with `.pad()`, `.expand_to_cube()`, etc.
- Extract image regions using `.crop_image()`.
- Create boxes from masks with `.from_mask_bbox()` or `.from_mask_centroid()`.

Try it out and make working with medical image regions easier!